# Psychology Modelling in Roulette

In this project i wish to discuss modelling the behaviours of different types of people given the value they place on money. In general, it is accepted that rich people place less value on money, since they have more of it. This in turn makes them more risk prone. The opposite applies to people who are poor. Decision theory provides us with a powerful tool in utility functions which allow us to model the risk attitude of different people with a function that is pre defined. We will delve into this in a bit more depth later onbut first we need to get started with defining the actual game of roulette.

In [3]:
import numpy as np
import random
import pandas as pd

These are the libraries which we use throughout so we import them first

## Creating The Roulette Class

In order to create the game of roulette, we define a class with different properties, for example the numbers in red are defined as a property and so on. Within this class we define both European roulette and American roulette.

In [4]:
class roulette_tables:
    def __init__(self, numbers_list):
        self.numbers_list=numbers_list
        self.red=[1,3,5,7,9,12,14,16,18,21,23,25,27,28,30,32,34,36]
        self.black=[2,4,6,8,11,15,17,20,29,22,24,26,29,31,33,35]
        self.green= [value for value in numbers_list if isinstance(value,str)]
        self.odd= [x for x in numbers_list if isinstance(x,int) and x%2!=0]
        self.even=[x for x in numbers_list if isinstance(x,int) and x%2==0]
        self.first_twelve=[1,2,3,4,5,6,7,8,9,10,11,12]
        self.second_twelve=[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
        self.third_twelve=[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
        self.one_eighteen=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
        self.nineteen_thirtysix=[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]

    def spin(self):
        number_outcome=np.random.randint(0,len(self.numbers_list))
        return(self.numbers_list[number_outcome])
    
    def show_odds(self):
        odds= {
    'Bet': ['Single Number', 'Odd/Even', 'Red/Black', 'Dozens'],
    'European probabilities': [1/37, 18/37, 18/37, 12/37],
    'American probabilities':[1/38,18/38,18/38,12/38],
    'payouts':[35,2,2,3]
        }
        odds=pd.DataFrame(odds)
        return(odds)
    
            
European_roulette=roulette_tables(['0',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36])
American_roulette=roulette_tables(['00','0',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36])
    

You will notice that the majority of the properties are hardcoded, which makes sense given the finite nature of the properties. Doing it this way may have taken longer to code but it will speed up the running of the code when using the functions we define later.

If you look closely, you will see that i have also defined methods inside the roulette class, namely the spin method, and the show_odds method. These 'do exactly what they say on the tin' for lack of a better way to describe it and they are demonstrated below.

In [3]:
European_roulette.spin()

12

In [4]:
American_roulette.spin()

4

In [5]:
European_roulette.show_odds()

,Bet,European probabilities,American probabilities,payouts
0,Single Number,0.027027,0.026316,35
1,Odd/Even,0.486486,0.473684,2
2,Red/Black,0.486486,0.473684,2
3,Dozens,0.324324,0.315789,3


We make use of the pandas dataframe to aesthetically display the data that we want, for ease of access both European_roulette.show_odds() and American_roulette.show_odds() provide the same dataframe.

Inspecting the data inside the table, it is clear to see you would be better off playing european roulette due to the fact that for every bet, the european roulette has a better chance of winning with the exact same payout as the american roulette

## Beginning to Model Some Behaviour 

In this section we essentially want to begin modelling the behaviour of people by creating a class with different attributes and methods that are relevant to our goal of roulette and behaviour modelling. You will see that we have used lambda functions to define an utility function as an attribute for the person class, alongside the method of calculating the utility they have for a set of gambles.

In decision theory, if you have the utility function of a person then you can calculate the utility of that gamble to the person. The higher the utility is the better the person regards that gamble. For instance if someone asked you to decide to either flip a coin for £10 (take gamble g1) or get £5 gauranteed (gamble g2) and your utility function was U(£x)=x then U(g1)=U(g2)= 0/2 + 10/2 = 5

In our model we describe a rich person as a person with a convex utility function, a risk neutral person as a person with the utility function U(£x)=x (linear) and a risk averse person as someone with a concave utility function. For interest and some comedic effect we have also modelled a fourth person whomst you may be familair with and will be introduced to later if you read on.

Below is the following class which models a persons behaviour and allows them to play roulette either once or n times.

In [5]:
class people:
    def __init__(self, utility_function,initialportfolio,max_bet_at_once):
        self.utility_function=utility_function
        self.initialportfolio=initialportfolio
        self.max_bet_at_once=max_bet_at_once
    
    def calc_utility(self,table,Amount_to_bet):
        if table == European_roulette:
            list_of_potential_earnings=Amount_to_bet*(table.show_odds()['payouts'].values)
            X=list(map(self.utility_function, list_of_potential_earnings))
            list_of_gamble_utilities=X*(table.show_odds()['European probabilities'].values)
            return pd.DataFrame({'Gambles':table.show_odds()['Bet'] , 'utility': list_of_gamble_utilities})
        else:
            list_of_potential_earnings=Amount_to_bet*(table.show_odds()['payouts'].values)
            X=list(map(self.utility_function, list_of_potential_earnings))
            list_of_gamble_utilities=X*(table.show_odds()['American probabilities'].values)
            return pd.DataFrame({'Gambles':table.show_odds()['Bet'] , 'utility': list_of_gamble_utilities})
    
    def Play_single_game(self,table,portfolio):
        portfolio=portfolio
        initial_portfolio=portfolio
        Amount_to_bet=min(0.01*portfolio, self.max_bet_at_once)
        vect_of_utilities=self.calc_utility(table, Amount_to_bet)
        index_of_best_decision=vect_of_utilities['utility'].idxmax()
        gambledecision=table.show_odds()['Bet'][index_of_best_decision]
        if gambledecision=='Single Number':
            final_value_decision=table.spin()
        elif gambledecision=='Odd/Even':
            final_value_decision=random.choice(['odd','even'])
        elif gambledecision=='Red/Black':
            final_value_decision=random.choice(['red','black'])
        elif gambledecision=='Dozens':
            final_value_decision=random.choice(['first_twelve','second_twelve','third_twelve'])
        
        outcome=table.spin()
        if final_value_decision in ('odd','even'):
            if final_value_decision=='odd':
                if outcome in table.odd:
                    initial_portfolio=portfolio
                    portfolio += Amount_to_bet
                    win='WINNER :)'
                else:
                    initial_portfolio=portfolio
                    portfolio -= Amount_to_bet
                    win='LOSER :('
                return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio], 'portfolio':[portfolio]}) 
            else:
                if outcome in table.even:
                     initial_portfolio=portfolio
                     portfolio += Amount_to_bet
                     win='WINNER :)'
                else:
                     initial_portfolio=portfolio
                     portfolio -= Amount_to_bet
                     win='LOSER :('  
                return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio],'portfolio':[portfolio]}) 
                     
        if final_value_decision in ('red','black'):
            if final_value_decision=='red':
                if outcome in table.red:
                    initial_portfolio=portfolio
                    portfolio += Amount_to_bet
                    win='WINNER :)'
                    
                else:
                    initial_portfolio=portfolio
                    portfolio -= Amount_to_bet
                    win='LOSER :('
                    
                return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio],'portfolio':[portfolio]}) 
            else:
                if outcome in table.black:
                     initial_portfolio=portfolio
                     portfolio += Amount_to_bet
                     win='WINNER :)'
                     
                else:
                     initial_portfolio=portfolio
                     portfolio -= Amount_to_bet
                     win='LOSER :('
                     
                return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio],'portfolio':[portfolio]}) 
                  
        if final_value_decision in ( 'first_twelve' , 'second_twelve' , 'third_twelve' ):
            if final_value_decision=='first_twelve':
                if outcome in table.first_twelve:
                    initial_portfolio=portfolio
                    portfolio += 3*Amount_to_bet
                    win='WINNER :)'
                    
                else:
                    initial_portfolio=portfolio
                    portfolio -= Amount_to_bet
                    win='LOSER :('
                    
                return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio], 'portfolio':[portfolio]}) 
            elif final_value_decision=='second_twelve':
                if outcome in table.second_twelve:
                    initial_portfolio=portfolio
                    portfolio += 3*Amount_to_bet
                    win='WINNER :)' 
                    
                else:
                    initial_portfolio=portfolio
                    portfolio -= Amount_to_bet
                    win='LOSER :('
                    
                return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio],'portfolio':[portfolio]}) 
            elif final_value_decision=='third_twelve':
                if outcome in table.third_twelve:
                    initial_portfolio=portfolio
                    portfolio += 3*Amount_to_bet
                    win='WINNER :)'
                    
                else:
                    initial_portfolio=portfolio
                    portfolio -= Amount_to_bet
                    win='LOSER :('
                    
                return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio], 'portfolio': [portfolio]} ) 
        
        
        else:
            if outcome==final_value_decision:
                initial_portfolio=portfolio
                portfolio+=34*Amount_to_bet
                win='WINNER :)'
                
            else:
                initial_portfolio=portfolio
                portfolio-=Amount_to_bet
                win='LOSER :('
                
            return pd.DataFrame({'outcome': [win], 'winnings': [portfolio-initial_portfolio],'portfolio':[portfolio]}) 
        
               
    def playing_n_times(self, n, table):
        portfolio=self.initialportfolio
        initial_portfolio=self.initialportfolio
        wins=0
        losses=0
        for i in range(n):
            if initial_portfolio==0:
                return 'You are now Broke!'
            else:
                Results=self.Play_single_game(table, portfolio)
                initial_portfolio=Results['portfolio'][0]
                if Results['outcome'][0]=='WINNER :)':
                    portfolio=portfolio+Results['winnings'][0]
                    wins+=1
                else:
                    portfolio=portfolio+Results['winnings'][0]
                    losses+=1
        return pd.DataFrame({'wins':[wins], 'losses':[losses],'portfolio':[portfolio]})
              
           
    

Its important to note some of the intricacies of the code provided, the first being that we assume that all of these people are reasonably seasoned gamblers. They do not risk more than 1% of their portfolio in any gamble. They also always place bets on the gamble which has the highest utility for them. for example, for a rich person the concept of winning small amounts of money is trivial, so they almost always bet on a single value in order to increase the potential earnings, the same applies to a poor person who always bets on red/black or odd/even. In every case a pandas dataframe is provided as the result when the games are played.

In [6]:
richperson=people(lambda x: 0.1*x**2, 100000,1000)
poorperson=people(lambda x:np.log(x+1),100,1)
neutralperson=people(lambda x: x,10000,100)

In [14]:
neutralperson.Play_single_game(European_roulette, neutralperson.initialportfolio)

bet placed: even outcome: 30


,outcome,winnings,portfolio
0,WINNER :),100.0,10100.0


In [15]:
richperson.Play_single_game(European_roulette, richperson.initialportfolio)

bet placed: 6 outcome: 7


,outcome,winnings,portfolio
0,LOSER :(,-1000.0,99000.0


In [16]:
poorperson.Play_single_game(European_roulette,poorperson.initialportfolio)

bet placed: even outcome: 3


,outcome,winnings,portfolio
0,LOSER :(,-1.0,99.0


So we can see in a single game setting a rich person has a higher utiltiy for single values whereas a more logical neutral and poor person are convinced by the higher odds of winning if you bet on the even/odd or red/black sections. Its important to note that these bets are consistant with the utilities calculated using the calc_utility function. As a demonstrarion these are also provided below.

In [17]:
neutralperson.calc_utility(European_roulette, neutralperson.max_bet_at_once)

,Gambles,utility
0,Single Number,94.594595
1,Odd/Even,97.297297
2,Red/Black,97.297297
3,Dozens,97.297297


In [18]:
richperson.calc_utility(European_roulette, richperson.max_bet_at_once)

,Gambles,utility
0,Single Number,3.310811e+06
1,Odd/Even,1.945946e+05
2,Red/Black,1.945946e+05
3,Dozens,2.918919e+05


In [19]:
poorperson.calc_utility(European_roulette, poorperson.max_bet_at_once)

,Gambles,utility
0,Single Number,0.096852
1,Odd/Even,0.534460
2,Red/Black,0.534460
3,Dozens,0.449609


## Erratic Behaviour

We are still yet to use the play_n_times function and also introduce the fourth person. Erratic behaviour can be modelled using randomness inside the utility function of the person we believe to be erratic. In this case, the most likely scene is to see someone get drunk at the casino and start placing bets with seemingly no logic whatsoever. We have added randomness in the form of shifting a sin function by a randomly chosen integer to provide illogical conclusions which differ every time you play a game.

In [10]:
drunkperson=people(lambda x: np.sin(x)**2+random.randint(-5, 5),10000,10000)

We are still assuming he is at the casino for a good night out and doesnt want to leave early  so he wont risk more than 1% of his portfolio in order to prolong his night out. lets play a few independent games to demonstrate the theory behind this.

In [22]:
drunkperson.Play_single_game(European_roulette, drunkperson.initialportfolio)

bet placed: even outcome: 24


,outcome,winnings,portfolio
0,WINNER :),100.0,10100.0


In [23]:
drunkperson.Play_single_game(European_roulette, drunkperson.initialportfolio)

bet placed: black outcome: 23


,outcome,winnings,portfolio
0,LOSER :(,-100.0,9900.0


In [24]:
drunkperson.Play_single_game(European_roulette, drunkperson.initialportfolio)

bet placed: first_twelve outcome: 30


,outcome,winnings,portfolio
0,LOSER :(,-100.0,9900.0


In [25]:
drunkperson.Play_single_game(European_roulette, drunkperson.initialportfolio)

bet placed: first_twelve outcome: 36


,outcome,winnings,portfolio
0,LOSER :(,-100.0,9900.0


And finally we show what the utilities could look like for this person, the deciding factor in how he places his bets.

In [26]:
drunkperson.calc_utility(European_roulette, drunkperson.max_bet_at_once)

,Gambles,utility
0,Single Number,0.053633
1,Odd/Even,-0.808197
2,Red/Black,-0.321710
3,Dozens,-1.412669


In [27]:
drunkperson.calc_utility(European_roulette, drunkperson.max_bet_at_once)

,Gambles,utility
0,Single Number,-0.081502
1,Odd/Even,1.137749
2,Red/Black,-1.294683
3,Dozens,-0.115371


Clearly we can see that the randomness has caused the utilities to be different with every game and hence has modelled the erratic behaviour.

## Simulating a full night gambling

Although doing this has been quite some fun, it wouldnt be very 
academic without some semblence of a conclusion, so lets consider
playing the roulette game n times recording wether or not the player
goes broke.

It is correct to say that the process of tracking the value of the 
portfolio at any time t where t is the amount of bets placed, is a simple
random walk on a line. If you would like to know more about these types
of stochastic processes, it would be a good idea to start learning the basics
of markov chains moving towards simple random walks such as the gamblers
ruin problem which this is closest too.

We first consider the rich person and how his portfolio progresses as n increases.

In [28]:
richperson.playing_n_times(5, European_roulette)

bet placed: 20 outcome: 5
bet placed: 24 outcome: 3
bet placed: 14 outcome: 10
bet placed: 11 outcome: 16
bet placed: 12 outcome: 20


,wins,losses,portfolio
0,0,5,95099.00499


In [29]:
richperson.playing_n_times(30, European_roulette)

bet placed: 13 outcome: 21
bet placed: 31 outcome: 31
bet placed: 18 outcome: 10
bet placed: 32 outcome: 31
bet placed: 25 outcome: 14
bet placed: 14 outcome: 29
bet placed: 8 outcome: 25
bet placed: 18 outcome: 33
bet placed: 32 outcome: 33
bet placed: 22 outcome: 23
bet placed: 28 outcome: 34
bet placed: 19 outcome: 26
bet placed: 2 outcome: 35
bet placed: 4 outcome: 23
bet placed: 5 outcome: 19
bet placed: 1 outcome: 0
bet placed: 24 outcome: 8
bet placed: 35 outcome: 17
bet placed: 35 outcome: 29
bet placed: 35 outcome: 28
bet placed: 15 outcome: 14
bet placed: 13 outcome: 23
bet placed: 11 outcome: 22
bet placed: 22 outcome: 4
bet placed: 25 outcome: 34
bet placed: 24 outcome: 3
bet placed: 30 outcome: 31
bet placed: 33 outcome: 19
bet placed: 3 outcome: 6
bet placed: 17 outcome: 32


,wins,losses,portfolio
0,1,29,104660.0


In [31]:
richperson.playing_n_times(50, European_roulette)

bet placed: 24 outcome: 13
bet placed: 33 outcome: 31
bet placed: 27 outcome: 28
bet placed: 35 outcome: 3
bet placed: 15 outcome: 20
bet placed: 16 outcome: 7
bet placed: 11 outcome: 7
bet placed: 29 outcome: 9
bet placed: 5 outcome: 9
bet placed: 6 outcome: 25
bet placed: 9 outcome: 24
bet placed: 11 outcome: 6
bet placed: 33 outcome: 14
bet placed: 11 outcome: 20
bet placed: 10 outcome: 5
bet placed: 7 outcome: 34
bet placed: 8 outcome: 19
bet placed: 21 outcome: 22
bet placed: 16 outcome: 2
bet placed: 1 outcome: 2
bet placed: 0 outcome: 36
bet placed: 35 outcome: 9
bet placed: 14 outcome: 18
bet placed: 35 outcome: 24
bet placed: 0 outcome: 32
bet placed: 30 outcome: 7
bet placed: 2 outcome: 0
bet placed: 32 outcome: 16
bet placed: 1 outcome: 1
bet placed: 18 outcome: 31
bet placed: 19 outcome: 3
bet placed: 24 outcome: 23
bet placed: 15 outcome: 0
bet placed: 10 outcome: 23
bet placed: 15 outcome: 3
bet placed: 17 outcome: 2
bet placed: 31 outcome: 23
bet placed: 31 outcome: 0
be

,wins,losses,portfolio
0,1,49,81900.065668


Now looking at the neutral person

In [32]:
neutralperson.playing_n_times(5, European_roulette)

bet placed: even outcome: 1
bet placed: odd outcome: 0
bet placed: odd outcome: 16
bet placed: even outcome: 25
bet placed: odd outcome: 8


,wins,losses,portfolio
0,0,5,9509.900499


In [33]:
neutralperson.playing_n_times(30, European_roulette)

bet placed: even outcome: 33
bet placed: even outcome: 6
bet placed: even outcome: 13
bet placed: even outcome: 2
bet placed: even outcome: 16
bet placed: odd outcome: 27
bet placed: odd outcome: 21
bet placed: even outcome: 8
bet placed: even outcome: 35
bet placed: odd outcome: 19
bet placed: even outcome: 10
bet placed: odd outcome: 23
bet placed: odd outcome: 6
bet placed: odd outcome: 27
bet placed: even outcome: 13
bet placed: even outcome: 29
bet placed: even outcome: 1
bet placed: odd outcome: 33
bet placed: even outcome: 4
bet placed: even outcome: 13
bet placed: even outcome: 34
bet placed: odd outcome: 4
bet placed: even outcome: 33
bet placed: even outcome: 2
bet placed: odd outcome: 30
bet placed: odd outcome: 30
bet placed: even outcome: 20
bet placed: even outcome: 8
bet placed: odd outcome: 29
bet placed: even outcome: 10


,wins,losses,portfolio
0,18,12,10597.980101


In [34]:
neutralperson.playing_n_times(50, European_roulette)

bet placed: even outcome: 25
bet placed: even outcome: 16
bet placed: odd outcome: 17
bet placed: odd outcome: 36
bet placed: even outcome: 7
bet placed: odd outcome: 30
bet placed: odd outcome: 0
bet placed: odd outcome: 24
bet placed: even outcome: 29
bet placed: odd outcome: 27
bet placed: odd outcome: 4
bet placed: odd outcome: 14
bet placed: odd outcome: 8
bet placed: odd outcome: 4
bet placed: even outcome: 7
bet placed: even outcome: 24
bet placed: odd outcome: 22
bet placed: odd outcome: 28
bet placed: odd outcome: 2
bet placed: odd outcome: 12
bet placed: second_twelve outcome: 7
bet placed: odd outcome: 34
bet placed: odd outcome: 1
bet placed: even outcome: 31
bet placed: odd outcome: 1
bet placed: even outcome: 0
bet placed: first_twelve outcome: 10
bet placed: even outcome: 3
bet placed: even outcome: 29
bet placed: odd outcome: 33
bet placed: odd outcome: 23
bet placed: odd outcome: 9
bet placed: odd outcome: 5
bet placed: even outcome: 3
bet placed: even outcome: 24
bet 

,wins,losses,portfolio
0,18,32,8844.429071


We skip the poorperson simulation for brevity as it is similar to the neutral person in the sense that they place similar bets. Finally we see the drunkpersons night.

In [35]:
drunkperson.playing_n_times(5, European_roulette)

bet placed: 28 outcome: 33
bet placed: red outcome: 22
bet placed: red outcome: 8
bet placed: even outcome: 20
bet placed: 36 outcome: 33


,wins,losses,portfolio
0,1,4,9702.019701


In [36]:
drunkperson.playing_n_times(30, European_roulette)

bet placed: odd outcome: 21
bet placed: black outcome: 2
bet placed: first_twelve outcome: 16
bet placed: third_twelve outcome: 5
bet placed: even outcome: 14
bet placed: first_twelve outcome: 31
bet placed: red outcome: 32
bet placed: second_twelve outcome: 7
bet placed: second_twelve outcome: 25
bet placed: odd outcome: 16
bet placed: red outcome: 14
bet placed: second_twelve outcome: 23
bet placed: third_twelve outcome: 22
bet placed: even outcome: 17
bet placed: red outcome: 27
bet placed: third_twelve outcome: 12
bet placed: first_twelve outcome: 5
bet placed: odd outcome: 28
bet placed: odd outcome: 3
bet placed: even outcome: 34
bet placed: odd outcome: 34
bet placed: even outcome: 35
bet placed: black outcome: 11
bet placed: even outcome: 19
bet placed: black outcome: 35
bet placed: black outcome: 11
bet placed: third_twelve outcome: 35
bet placed: odd outcome: 20
bet placed: odd outcome: 24
bet placed: odd outcome: 9


,wins,losses,portfolio
0,15,15,10590.002886


In [12]:
drunkperson.playing_n_times(100, European_roulette)

,wins,losses,portfolio
0,32,68,7656.961233


## Conclusion

Although these portfolios at the end of the night do demonstrate the random nature of a simple random walk on a line, it is worth noting that since the probability of winning is always less then 49% as n tends to infinity, By the theory behind gamblers ruin, you will go broke at some point. In our case our portfolios would tend towards zero because of the way we are always using 1% of our portfolios, however in a real situation at some point we would be under the minimum bet on the table and hence we would be broke. The following code demonstrates this idea, if you allow the printing to finish and scroll to the end you will see the portfolio being extremely low while n is high. the disparity between wins and losses is high enough to cause the player to go broke.

In [8]:
poorperson.playing_n_times(10000, European_roulette)

,wins,losses,portfolio
0,4807,5193,1.32672


Thanks for reading my small project!  i hope it all made sense

if you have any questions please email me at callumhurd@hotmail.co.uk